In [1]:
import sys, os
sys.path.append('../../dependencies/')
import mfsetup
import flopy as fp
import flopy.utils as fu
from flopy.utils.gridintersect import GridIntersect

import shapefile as sf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
from shapely.geometry import Point, LineString
import geopandas as gp
datapath = Path('../../data/sgn/')

/Users/mnfienen/Documents/GIT/gis-utils/gisutils/__init__.py:14: DeprecationWarning: The 'project' module was renamed to 'projection' to avoid confusion with the project() function.
  warnings.warn("The 'project' module was renamed to 'projection' "
/Users/mnfienen/Documents/GIT/gis-utils/gisutils/__init__.py:14: DeprecationWarning: The 'project' module was renamed to 'projection' to avoid confusion with the project() function.
  warnings.warn("The 'project' module was renamed to 'projection' "
/Users/mnfienen/Documents/GIT/gis-utils/gisutils/__init__.py:14: DeprecationWarning: The 'project' module was renamed to 'projection' to avoid confusion with the project() function.
  warnings.warn("The 'project' module was renamed to 'projection' "
/Users/mnfienen/Documents/GIT/gis-utils/gisutils/__init__.py:14: DeprecationWarning: The 'project' module was renamed to 'projection' to avoid confusion with the project() function.
  warnings.warn("The 'project' module was renamed to 'projection' "


## one-liner to build the model

In [2]:
# note that modflow setup changes the directory, so good idea to change back at the end
stdir = os.getcwd()

In [3]:
m = mfsetup.MF6model.setup_from_yaml('sgn_config.yml')

loading configuration file sgn_config.yml...

Setting up sgn_50 model from configuration in sgn_config.yml

validating configuration...
DIS package
done with validation.

setting up model grid...
wrote /Users/mnfienen/Documents/GIT/SYMPLE/symple_flopy/models/sgn_mfsetup/sgn_50_grid.json
writing /Users/mnfienen/Documents/GIT/SYMPLE/symple_flopy/models/sgn_mfsetup/postproc/shps/sgn_50_bbox.shp... Done
finished in 0.14s


Setting up TDIS package...
finished in 0.01s


Setting up IMS package...
finished in 0.00s


Setting up DIS package...
reading data from /Users/mnfienen/Documents/GIT/SYMPLE/symple_flopy/data/sgn/raster/DTM_domain.tif...
finished in 0.07s
wrote ././top.dat, took 0.00s
reading data from /Users/mnfienen/Documents/GIT/SYMPLE/symple_flopy/data/sgn/raster/Bott_L1_fix.tif...
finished in 0.13s
reading data from /Users/mnfienen/Documents/GIT/SYMPLE/symple_flopy/data/sgn/raster/Bott_L2_fix.tif...
finished in 0.12s
reading data from /Users/mnfienen/Documents/GIT/SYMPLE/symple_flop

In [4]:
m.write_input()

writing simulation...
  writing simulation name file...
  writing simulation tdis package...
  writing ims package ims_-1...
  writing model sgn_50...
    writing model name file...
    writing package dis...
    writing package ic...
    writing package npf...
    writing package rcha_0...
    writing package oc...
    writing package sfr_0...
    writing package wel_0...
INFORMATION: maxbound in ('gwf6', 'wel', 'dimensions') changed to 78 based on size of stress_period_data
SFRmaker v. 0.6.2.post46+g588f7f2

Running Flopy v. 3.3.5 diagnostics...
passed.

Checking for continuity in segment and reach numbering...
passed.

Checking for increasing segment numbers in downstream direction...
passed.

Checking for circular routing...
passed.

Checking reach connections for proximity...
0 segments with non-adjacent reaches found.
At segments:


0 segments with non-adjacent reaches found.
At segments:



Checking for model cells with multiple non-zero SFR conductances...
8 model cells with mu

### set up an intersection object

In [ ]:
ix = GridIntersect(m.modelgrid)

## need to add a GHB package - manually for now

In [ ]:
# read in the boundary polygon
boundary = gp.read_file(datapath / 'shp' / 'Model_domain.shp')

# create a polyline from the polygon
boundary_line =  LineString([Point(i) for i in boundary.iloc[0].geometry.exterior.coords])
boundary_line

In [ ]:
# instersect the bounding polygon with the grid to get boundary cells
ix_boundary = ix.intersect(boundary_line.buffer(25, resolution=4))

In [ ]:
ix_df = pd.DataFrame.from_records(ix_boundary)
ix_df.head()

In [ ]:
# read in the september 2019 measured heads to make the GHB data
sep19_head = fu.Raster.load(datapath / 'raster' / 'heads_sep2019.asc')
plt.imshow(sep19_head.get_array(1))
plt.colorbar()

In [ ]:
# resample these heads to the model
head = sep19_head.resample_to_grid(m.modelgrid,band=sep19_head.bands[0], method="nearest")

In [ ]:
# make the GHB stress period data
ghb_spd = []
for icpl in ix_boundary.cellids:
    cond = 100.0
    for layer in [0,2]:
        # note - structured grid so *icpl unpacks row/col
        ghb_spd.append([(layer, *icpl), head[icpl], cond, f'GHB{layer+1}'])
       

In [ ]:
# create the package
ghb = fp.mf6.ModflowGwfghb(model=m, 
                                stress_period_data=ghb_spd, 
                                boundnames=True) 

In [ ]:
m.write_input()

In [ ]:
m

In [ ]:
m.dis.export('testgrid.shp')

In [ ]:
#os.system('mf6')

In [ ]:
top = m.dis.top.array

In [ ]:
plt.imshow(top)

In [ ]:
h = fu.binaryfile.HeadFile('../../models/sgn_mfsetup/sgn_50.hds').get_alldata()

In [ ]:
plt.imshow(np.squeeze(h)[0])
plt.colorbar()

### check flooding

In [ ]:
plt.imshow(top-np.squeeze(h)[0])
plt.colorbar()

In [ ]:
df1,df2 = fu.Mf6ListBudget('../../models/sgn_mfsetup/sgn_50.list').get_dataframes()

In [ ]:
df1

In [ ]:
df2